<a href="https://colab.research.google.com/github/yota-nagai/combinatorial-optimization-problems/blob/main/06_%E6%9C%80%E5%B0%8F%E6%A5%B5%E5%A4%A7%E3%83%9E%E3%83%83%E3%83%81%E3%83%B3%E3%82%B0%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q amplify
from amplify import VariableGenerator, FixstarsClient, solve, Solver
from datetime import timedelta

client = FixstarsClient()
client.token = "******************"  #アクセストークンを入力
client.parameters.timeout = timedelta(milliseconds=2000)  # タイムアウトは 2000 ms


#################### 最小極大マッチング問題 ####################

#グラフの情報を入力
N,M=map(int,input().split()) #頂点数、辺の数
edge=[]  #辺リスト（無向グラフ）
for _ in range(M):
  u,v=map(int,input().split())
  u-=1
  v-=1
  edge.append([u,v])
  # [v,u])を加えてはダメ　全頂点ペアではなく、辺リストを巡回する

#変数の生成
gen=VariableGenerator()
q=gen.array("Binary",M)  #辺iを選ぶかどうか
y=[0]*N  #頂点iがいくつの辺によって被覆されるか
for i in range(N):
  for j in range(M):
    u,v=edge[j]
    if u==i:
      y[i]+=q[j]
    elif v==i:
      y[i]+=q[j]

#制約項①　マッチングになっているか
f1=0
for i in range(N):
  f1+=y[i]*(y[i]-1)

#制約項②　極大性について
f2=0
for u,v in edge:
  f2+=(1-y[u])*(1-y[v])

#目的関数項　えらぶ辺の数の最小化
f3=0
f3=sum(q)

#ペナルティ係数を設定
A,B,C=map(int,input().split())
f=A*f1+B*f2+C*f3


#ソルバー実行、最適解の取得
result=solve(f,client)
objective=result.best.objective
values=list(result.best.values.values())


#エネルギー関数を最小にする辺の選び方を出力
print(f"エネルギー関数の最小値 = {objective}")
chosen=[]
for i in range(M):
  if values[i]==1:
    chosen.append((edge[i][0]+1,edge[i][1]+1))
print(chosen)